### KNU 한국어 감성사전 - DI LAB
참조: http://dilab.kunsan.ac.kr/knusl.html

In [1]:
import json
import pandas as pd
from tqdm import tqdm

In [5]:
# KNU 한국어 감성사전 파일 읽어오기
SentiWord = pd.read_json("SentiWord_info.json")
SentiWord.head(5)
SentiWord.tail(5)

,word,word_root,polarity
14838,반신반의하다,반신반의,0
14839,신비롭다,신비,1
14840,아리송하다,아리송,-1
14841,알쏭하다,알쏭하,-1
14842,알쏭달쏭하다,알쏭달쏭,-1


In [7]:
# SentiWord에 새로운 단어 추가 및 csv 형식으로 저장
SentiWord=SentiWord.append({"word":"알쏭", "word_root":"알쏭", "polarity":-1}, ignore_index=True)
SentiWord.to_csv("SentiWord_info.csv", index=False)
SentiWord.tail()

,word,word_root,polarity
14840,아리송하다,아리송,-1
14841,알쏭하다,알쏭하,-1
14842,알쏭달쏭하다,알쏭달쏭,-1
14843,알쏭,알쏭,-1
14844,알쏭,알쏭,-1


In [8]:
#SentiWord=SentiWord.drop(14844)
#SentiWord

In [10]:
SentiWord = pd.read_csv("SentiWord_info.csv")
SentiWord

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14840,아리송하다,아리송,-1
14841,알쏭하다,알쏭하,-1
14842,알쏭달쏭하다,알쏭달쏭,-1
14843,알쏭,알쏭,-1


In [11]:
SentiWord.loc[len(SentiWord)]=["합의", "합의", 1]
SentiWord.loc[len(SentiWord)]=["징용", "징용", -2]
SentiWord.tail()

,word,word_root,polarity
14842,알쏭달쏭하다,알쏭달쏭,-1
14843,알쏭,알쏭,-1
14844,알쏭,알쏭,-1
14845,합의,합의,1
14846,징용,징용,-2


In [12]:
# 단어 입력후 긍/부정 전달
def pos_neg(word):
    tmp=SentiWord[(SentiWord['word']==word) | (SentiWord['word_root']==word)]
    try:
        word_res=(word, tmp['polarity'][tmp.index[0]])
    except:
        word_res=(word, 0)
        
    return word_res

In [13]:
word_res = pos_neg("합의")
print(word_res)

('합의', 1)


### 영화 리뷰 평가

In [14]:
move_df = pd.read_csv('move_reple.csv')
move_df.head()

,별점,리뷰,날짜
0,10,최근 본 영화중에 시계 안본 영화는 간만이네요. 시간이 순삭이네요,2021.11.24
1,10,"윤계상이 목숨 걸었다고 하던데, 확실히 액션이 돋보임. 카체이싱도 아찔하고. 박용우...",2021.11.24
2,8,시사회로 먼저 봤는데 난 재밌게 봄 배우들 연기도 좋고 마지막 액션씬도 좋았음 박지...,2021.11.24
3,10,"윤계상 연기 너무 잘하고, 액션도 시원시원하구 내용도 신선해서 재밌었음",2021.11.24
4,8,기대안하고 갔는디 생각보다 많이재밌었어요,2021.11.24


In [15]:
# 리뷰 하나의 텍스트로 만들기

text =""
for i in move_df.index:
    text += str(move_df.loc[i, '리뷰']) + "\n"
    
print(text[:50])

최근 본 영화중에 시계 안본 영화는 간만이네요. 시간이 순삭이네요 
윤계상이 목숨 걸었다고


In [17]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 384 kB/s 
     |████████████████████████████████| 448 kB 13.1 MB/s 


In [ ]:
# Konlp를 이용한 형태소 분석 진행후 morps에 자장
from konlpy.tag import Okt
okt = Okt()

morps = okt.morphs(text)
morps

In [19]:
from tqdm import tqdm

pos_list=[]   # 긍정 단어
neg_list=[]   # 부정 단어
unkwon_list=[]# 중립 단어

for noun in tqdm(morps, "긍/부정 나누기"):
    word_res=pos_neg(noun)
    if word_res[1] > 0:
        pos_list.append(word_res[0])
    elif word_res[1] < 0:
        neg_list.append(word_res[0])
    else:
        unkwon_list.append(word_res[0])

긍/부정 나누기: 100%|██████████| 2887/2887 [00:10<00:00, 267.84it/s]


In [21]:
print(f"긍정 키워드 건수: {len(pos_list)} 개")
print(f"부정 키워드 건수: {len(neg_list)} 개")
print(f"중립/모르는 키워드 건수:{len(unkwon_list)} 개")

긍정 키워드 건수: 196 개
부정 키워드 건수: 270 개
중립/모르는 키워드 건수: 2421 개


In [ ]:
stop_words ="것 넘 점 ( 와 히 로 위 시 사람 순 타 보 열 ) 이 때 급 저 아 가"
stop_words = stop_words.split()

# stop_words를 이용한 단어 정리
newPos_list=[]

for w in pos_list:
    if w not in stop_words:
        newPos_list.append(w)

newPos_list[:50]

In [ ]:
# 한 글자 제거
newNeg_list=[]

for w in neg_list:
    if len(w) > 1 :
        newNeg_list.append(w)

newNeg_list[:50] 

In [31]:
print(f"긍정 키워드 건수: {len(newPos_list)} 개")
print(f"부정 키워드 건수: {len(newNeg_list)} 개")

긍정 키워드 건수: 159 개
부정 키워드 건수: 40 개


In [ ]:
from collections import Counter

pos_count=dict(Counter(newPos_list).most_common())  #단어별 빈도수 계산 후 내림차순 정렬
neg_count=dict(Counter(newNeg_list).most_common())

pos_count

In [41]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [43]:
img_path="good.jpg"
img=Image.open(img_path)
img.size

(831, 789)

In [44]:
cloud_mask=np.array(Image.open(img_path))
print(cloud_mask.shape)
#print(cloud_mask)

(789, 831, 3)


색상 참조: https://www.w3schools.com/colors/colors_hexadecimal.asp

In [46]:
def color_func1(**kawargs):
    color="#cf513c"
    return color

In [ ]:
wordcloud = WordCloud(font_path="c:/Windows/Fonts/malgun.ttf",
                      background_color="white", mask=cloud_mask)

wc = wordcloud.generate_from_frequencies(pos_count)

plt.figure(figsize=(10,15))
plt.imshow(wc.recolor(color_func=color_func1), interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# 부정을 워드클라우드로 출력

cloud_mask=np.array(Image.open('bad.jpg'))
print(cloud_mask.shape)

wordcloud = WordCloud(font_path="c:/Windows/Fonts/malgun.ttf",
                      background_color="white", mask=cloud_mask)

wc = wordcloud.generate_from_frequencies(neg_count)

plt.figure(figsize=(10,15))
plt.imshow(wc.recolor(color_func=color_func1), interpolation='bilinear')
plt.axis("off")
plt.show()